# `II. Field Simulation` 
This .pynb file reads `./inter_results/mesh_result.pkl` generated by `./I_Mesh_Processing.ipynb`, and simulates electric field distribution. The result is stored in `./inter_results/field_result.pkl`.

In this file we compute electric field or potential on some spatial points(called `grid`) for all electrode voltage configurations(i.e. one eletrode is 1V and the rest 0V).Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.


In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

notes:
* here we invoke `multiprocessing.set_start_method("fork")` for compatibility with python version 3.9
* This is the most time consuming part in our workflow. Running following codes in a `.py` file instead of `.ipynb` may be helpful. Closing other apps in your laptop is helpful. Using HPC is also an option. 
* for reference, this file in my laptop(macbook pro, 2 physical kernels) runs 800s
* This `.ipynb` file will generate a `./.vtk` file for intermediate files, which can be simply ignored.

## (1) import the python packages needed for the script



In [1]:
from time import time
import pickle
import numpy as np
import os
import pyvista as pv
import sys
import vtk
sys.path.append('/home/sqip/Documents/github/bem')
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
import multiprocessing 
# multiprocessing.set_start_method("fork")
from utils.helper_functions import *

## (2) import the mesh saved from I_Mesh_Processing
This should correspond to the same name was exported during mesh_processing. The reason this is even an option is in case you want to load in a different saved mesh. 

In [2]:
radius= 500e-3
area = 1e-3
file = 'mit_LL'
file_in_name = 'inter_results/inter_results_mit/'+file+'_'+str(radius)+'_'+str(area)+'.pkl'
vtk_out = "inter_results/inter_results_mit/.vtks/"+file
file_out_name = 'inter_results/inter_results_mit/'+file+'_'+str(radius)+'_'+str(area)+'_simulation'


#open the mesh that will be used for simulation
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing

## (2) define grid
Here, we define some spatial points for simulation field, called 'grid'.
The grid is typically a 3-d cube centered on the trapping location.
* s is the unit step for the spatial points
* Lx, Ly, Lz are the dimensions of the 3-d cube
* sx, sy, sz are the step size along each dimension
* nx, ny, nz are the number of points to be simulated along each dimension
* `Grid` is the function imported from the bem package that will be inputted into the simluation

In [3]:
s = 1e-3
# L = 102e-6
Lx, Ly, Lz = 15*1e-3,15e-3,15*1e-3# in the unit of scaled length mesh_unit
# xl,yl,zl = -3.75*1e-3,72*1e-3,270*1.0e-3
xl,yl,zl = 0.0e-3,50.0e-3,0.0e-3
sx,sy,sz = s,s,s
print("done")
# ni is number of grid points, si is step size. To  on i direction you need to fix ni*si.
nx, ny, nz = [int(Lx/sx),int(Ly/sy),int(Lz/sz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)

grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx,ny,nz))

# Grid center (nx, ny ,nz)/2 is shifted to origin
print("lowval",grid.indices_to_coordinates([0,0,0]))
print("Grid center index", grid.indices_to_coordinates([nx/2,ny/2,nz/2]))
print("total points to be simulated:",nx*ny*nz)


done
Size/l: 0.015 0.015 0.015
Step/l: 0.001 0.001 0.001
Shape (grid point numbers): 15 15 15
lowval [-0.007  0.043 -0.007]
Grid center index [0.0005 0.0505 0.0005]
total points to be simulated: 3375


## (2) run jobs
`pmap.multiprocessing.Pool(processes).map(job,list)` is used to do several simulations in parallel (depends what you define as parallel). 

`Pool(number)` creates a set of worker processes, called a pool, to submit tasks to. When a job is submitted to a worker process, the worker process executes that task. <b>Each worker process is associated with its own CPU core</b> which means the number of worker processes in the pool is the number of parallel computations that can be performed. <b>number</b> is the number of worker processes to start. 

In other words, if you want to use only 1 core at a time, write Pool(1), if you want 2 cores write Pool(2). If no number is given, `Pool` will create as many worker processes as there are cores on the computer. This speeds things up (you can use all cores simulatneously), but it is important to point out that since each worker process will be requesting RAM, this will consume more memory on your computer than Pool(1). If the total RAM requested exceeds what your computer has, <b>you're gonna have a bad time</b>. One of two things will likely happen- first is that your computer will try to start using 'swap space' which is using disk memory as RAM which will slow down the computation significantly- this is not obvious unless you are looking at the resources being used by your computer (look for normal memory used and swap memory used on system monitor). Second is that your computer wil crash because the worker processes have consumed all the memory. 

`Pool.map`(<b>job</b>,<b>jobs_list</b>) takes two arguments:
* <b>job</b> is the python function that will be turned into a job to be executed by the worker processes
* <b>jobs_list</b> is a list where each item in the list is a set of arguments to be provided to the function when creating a job

The worker processes then execute any job sent to the pool. 

In [4]:

jobs = list(Configuration.select(mesh,"DC.*","RF"))
# run the different electrodes on the parallel pool
pmap = multiprocessing.Pool(8).map # parallel map
#pmap = map # serial map
t0 = time()
# range(len(jobs))
def run_map():
    jobs_list = ((jobs[i], grid, vtk_out,i,len(jobs)) for i in np.arange(len(jobs)))
    list(pmap(run_job,jobs_list))
    print("Computing time: %f s"%(time()-t0))
    # run_job casts a word after finishing ea"ch electrode.

run_map()


starting job 1 out of 21
starting job 2 out of 21
starting job 3 out of 21
starting job 4 out of 21starting job 5 out of 21

starting job 6 out of 21
starting job 7 out of 21
starting job 8 out of 21
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.04131
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.04131
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.45661
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.76407
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.76407
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.44088
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, c

/home/sqip/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:508: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


starting job 9 out of 21
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.6

/home/sqip/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:508: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


starting job 10 out of 21
finished job DC14
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148


...

starting job 14 out of 21
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
FLW-placeq: oversized panel, cube length=2.51968 panel length=2.62148
finished job DC13


/home/sqip/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:508: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


starting job 15 out of 21
finished job DC20


/home/sqip/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:508: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


starting job 16 out of 21
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.04131
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.04131
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.45661
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.76407
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.76407
FLW-placeq: oversized panel, cube length=1.25984 panel length=1.44088
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.22385
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.15511
FLW-placeq: oversized panel, cube length=1.25984 panel length=2.

## (3) view simluation results
The results can be viewed in an interactive 3-d plot using the package pyvista

In [4]:
"the last simulated mesh was for mesh radius 500e-3mm and max triangle area 3.125e-5 mm"
ele = 'DC5'
data_name = "%s_%s.vtk" % (vtk_out, ele)
data = pv.UniformGrid(data_name)
scalar_name = 'potential'
avals = data[scalar_name]
input_range = [avals.min(),avals.max()]
Result.view(vtk_out, ele, rng = input_range) # add electrode name between '' for result view

In [5]:
# for x in np.arange(0,4):
data_name = "%s_%s.vtk" % (vtk_out, 'RF')
import pyvista as pv 
data = pv.UniformGrid(data_name)
scalar_name = 'potential'

avals = data[scalar_name]
input_range = [avals.min(),avals.max()*5]
Result.view(vtk_out, 'RF',isosurfaces=100, rng=  input_range) # add electrode name between '' for result view

ValueError: Empty meshes cannot be plotted. Input mesh has zero points.

## (4) save simulation results

In [ ]:
write_pickle(vtk_out,file_out_name,grid,electrode_names)

KeyboardInterrupt: 